## steamspypi 5000개 데이터 수집 & 전처리

In [3]:
import pandas as pd
import numpy as np
import steamspypi
from datetime import datetime
import time

In [ ]:
def get_steamspy_data(N):

    lst = []
    
    # steamspypi를 이용해 보유자 수 상위 N*1000개의 데이터 수집 & 데이터프레임 형태로 저장
    for i in range(N):
        data_request = {'request' : 'all', 'page' : f'{i}'} 
        data = steamspypi.download(data_request)
        
        temp_df = (pd.DataFrame.from_dict(data, orient = 'index') # 'index' 지정해야 인덱스 별로 행-열이 저장됨
                  .reset_index()
                  .drop('index', axis = 1)
         )
        lst.append(temp_df)
        print(f"{(i + 1) * 1000}개 데이터 수집 완료")
        
        # request = all 요청은 1분에 1번씩만 가능
        time.sleep(60)
    
    
    df = pd.concat(lst) 
    return df

n = 3 # 3000개 수집
today = datetime.now().date().strftime('%Y-%m-%d')

df = get_steamspy_data(n)

1000개 데이터 수집 완료


---
- 이슈 : **steamspy의 모든 데이터를 가져오는 요청이 정상적으로 작동하지 않고 있음**
- 예전에 3월 27일 기준으로 수집한 데이터가 있으니 여기에 있는 데이터만을 대상으로 진행해봄

In [104]:
df = pd.read_csv('230327_SteamTop5000.csv')
df = df.drop(['Unnamed: 0', 'appid_y'], axis = 1)
df = df.rename(columns = {'appid_x' : 'app_id'})

In [105]:
print(df['app_id'].nunique()) # 4844
print(df.shape[0])

4844
4864


## 1. 전처리

### 중복값 확인 후 제거
- `app_id`를 PK로 취급함
- 실제로 스팀에 검색했을 때 같은 이름의 게임이어도 여러 데이터가 나오지 않기 때문에 취합과정에서 생긴 오류이거나 steamspy 자체의 오류 같음
- 겹치는데 다른 값들이 있다면 1번째 데이터만 살림

In [69]:
# PK에 쓰기 좋은 코드일 듯

def get_dupl(df, col: str):
    """
    고유해야 하는 column에 적용, 중복값의 상태를 살펴봄
    """
    dupl = df[col].value_counts() >= 2
    dupl_df = df[df[col].map(dupl)]
    return dupl_df

In [106]:
df = df.drop_duplicates() # 모든 데이터가 완전히 겹치는 경우 제외

get_dupl(df, 'app_id')

,app_id,name,developer,publisher,positive,negative,owners,average_forever,average_2weeks,median_forever,...,tag_Instrumental Music,tag_Electronic Music,tag_Birds,tag_Documentary,tag_Feature Film,tag_Steam Machine,tag_Fox,tag_Shop Keeper,tag_Hobby Sim,tag_Baseball
142,22380,Fallout: New Vegas,Obsidian Entertainment,Bethesda Softworks,169217,6212,5m ~ 10m,3615,316,1054,...,0,0,0,0,0,0,0,0,0,0
143,22380,Fallout: New Vegas,Obsidian Entertainment,Bethesda Softworks,169217,6212,5m ~ 10m,3615,316,1054,...,0,0,0,0,0,0,0,0,0,0
144,22490,Fallout: New Vegas,Obsidian Entertainment,Bethesda Softworks,46753,2153,500k ~ 1m,2950,403,1626,...,0,0,0,0,0,0,0,0,0,0
145,22490,Fallout: New Vegas,Obsidian Entertainment,Bethesda Softworks,46753,2153,500k ~ 1m,2950,403,1626,...,0,0,0,0,0,0,0,0,0,0
471,783770,Ironsight,"WipleGames Inc.,","WipleGames Inc.,",14306,5389,2m ~ 5m,891,78,87,...,0,0,0,0,0,0,0,0,0,0
472,783770,Ironsight,"WipleGames Inc.,","WipleGames Inc.,",14306,5389,2m ~ 5m,891,78,87,...,0,0,0,0,0,0,0,0,0,0
473,715220,Ironsight,Wiple Games,Aeria Games,4304,1178,1m ~ 2m,154,0,76,...,0,0,0,0,0,0,0,0,0,0
474,715220,Ironsight,Wiple Games,Aeria Games,4304,1178,1m ~ 2m,154,0,76,...,0,0,0,0,0,0,0,0,0,0
821,1509960,PICO PARK,TECOPARK,TECOPARK,17171,1574,1m ~ 2m,182,43,151,...,0,0,0,0,0,0,0,0,0,0
822,1509960,PICO PARK,TECOPARK,TECOPARK,17171,1574,1m ~ 2m,182,43,151,...,0,0,0,0,0,0,0,0,0,0


In [107]:
get_dupl(df, 'name')

,app_id,name,developer,publisher,positive,negative,owners,average_forever,average_2weeks,median_forever,...,tag_Instrumental Music,tag_Electronic Music,tag_Birds,tag_Documentary,tag_Feature Film,tag_Steam Machine,tag_Fox,tag_Shop Keeper,tag_Hobby Sim,tag_Baseball
142,22380,Fallout: New Vegas,Obsidian Entertainment,Bethesda Softworks,169217,6212,5m ~ 10m,3615,316,1054,...,0,0,0,0,0,0,0,0,0,0
143,22380,Fallout: New Vegas,Obsidian Entertainment,Bethesda Softworks,169217,6212,5m ~ 10m,3615,316,1054,...,0,0,0,0,0,0,0,0,0,0
144,22490,Fallout: New Vegas,Obsidian Entertainment,Bethesda Softworks,46753,2153,500k ~ 1m,2950,403,1626,...,0,0,0,0,0,0,0,0,0,0
145,22490,Fallout: New Vegas,Obsidian Entertainment,Bethesda Softworks,46753,2153,500k ~ 1m,2950,403,1626,...,0,0,0,0,0,0,0,0,0,0
243,202970,Call of Duty: Black Ops II,Treyarch,Activision,35191,5655,2m ~ 5m,340,7,125,...,0,0,0,0,0,0,0,0,0,0
245,202990,Call of Duty: Black Ops II,Treyarch,Activision,27100,4790,2m ~ 5m,4455,16,903,...,0,0,0,0,0,0,0,0,0,0
471,783770,Ironsight,"WipleGames Inc.,","WipleGames Inc.,",14306,5389,2m ~ 5m,891,78,87,...,0,0,0,0,0,0,0,0,0,0
472,783770,Ironsight,"WipleGames Inc.,","WipleGames Inc.,",14306,5389,2m ~ 5m,891,78,87,...,0,0,0,0,0,0,0,0,0,0
473,715220,Ironsight,Wiple Games,Aeria Games,4304,1178,1m ~ 2m,154,0,76,...,0,0,0,0,0,0,0,0,0,0
474,715220,Ironsight,Wiple Games,Aeria Games,4304,1178,1m ~ 2m,154,0,76,...,0,0,0,0,0,0,0,0,0,0


In [108]:
# app_id, name이 겹친다면 1번째만 남기고 나머지 제거함
df = df.drop_duplicates(subset = ['app_id'])
df = df.drop_duplicates(subset = ['name'])

In [109]:
# 최종 확인
print(df.shape[0])
print(df['app_id'].nunique())
print(df['name'].nunique())

4837
4837
4837


### 피처 정리

1. `tag` 피쳐 제거 : 이전에 했던 전처리 과정에서 tag를 붙여서 원핫인코딩한 피쳐를 만들었는데, 너무 많아서(400개 이상) 제거
2. `lang_` 피쳐 간소화 : 한국어와 일본어만 남김. 영어는 99.7%의 게임에서 지원하므로 모든 게임에서 지원한다고 간주함.
3. `owners` 제거 : 범위의 중간값을 `median_owner`로 얻고, 이 특성은 제거
4. `price` : 날짜에 따라 할인이 적용돼서 값이 바뀔 수 있으므로 제거
5. `category` : 

In [110]:
df = df[df.columns.drop(list(df.filter(regex = 'tag')))][:]

### 언어 피처 간소화
1. 스팀의 99.7%가 영어를 지원하므로 의미가 없어서 제거
2. 한국어랑 일본만 남기고 나머지 언어들 제거

In [111]:
df = df[df.columns.drop(list(df.filter(regex = 'lang')
                        .drop(['lang_Korean', 'lang_Japanese'], axis = 1)))]

### 기타 피처 제거
- `owners` : 범위의 중간값으로 `median_owner`를 얻은 다음 제거함
- `price` : 할인이 적용되면 가격이 변동되는데, 그날그날 가격이 달라지므로 제거

In [113]:
df = df.drop(['owners', 'price'], axis = 1)

In [138]:
df[list(df.filter(regex = 'category_'))].sum()


category_Gore                      593
category_Movie                      12
category_Photo Editing               4
category_Video Production           13
category_Web Publishing             10
category_Action                   3096
category_Adventure                2673
category_Animation & Modeling       24
category_Audio Production            6
category_Casual                   1469
category_Design & Illustration      32
category_Early Access              594
category_Education                  88
category_Free to Play              972
category_Game Development           24
category_Indie                    2843
category_Massively Multiplayer     520
category_Nudity                    345
category_RPG                      1496
category_Racing                    222
category_Simulation               1338
category_Software Training          10
category_Sports                    260
category_Strategy                 1613
category_Utilities                  52
category_Violent         

       app_id                               name  \
0         570                             Dota 2   
1         730   Counter-Strike: Global Offensive   
2     1172470                       Apex Legends   
3      578080                PUBG: BATTLEGROUNDS   
4     1063730                          New World   
...       ...                                ...   
4859   297120                      Heavy Bullets   
4860   319570  Heroes of SoulCraft - Arcade MOBA   
4861  1398210                 Selene ~Apoptosis~   
4862   425220            One Piece Burning Blood   
4863   402980                             Osteya   

                             developer                   publisher  positive  \
0                                Valve                       Valve   1612833   
1     Valve, Hidden Path Entertainment                       Valve   6252050   
2                Respawn Entertainment             Electronic Arts    515808   
3                        KRAFTON, Inc.               KR